In [ ]:
"""
Excel Model Optimizer - Phase 2: Review and Cleanup
Comprehensive cleanup and optimization tool for Excel financial models
"""

In [ ]:
import xlwings as xw
import openpyxl
import re
import json
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple, Set
from dataclasses import dataclass
from collections import defaultdict, Counter
from datetime import datetime
import pandas as pd

In [ ]:
@dataclass
class OptimizationIssue:
    """Container for optimization issues found"""
    category: str
    severity: str  # 'HIGH', 'MEDIUM', 'LOW'
    location: str
    description: str
    recommendation: str
    impact: str
    
    def to_dict(self) -> Dict:
        return {
            'category': self.category,
            'severity': self.severity,
            'location': self.location,
            'description': self.description,
            'recommendation': self.recommendation,
            'impact': self.impact
        }

In [ ]:
class ExcelModelOptimizer:
    """
    Comprehensive Excel Model Cleanup and Optimization Tool
    """
    
    def __init__(self, excel_file_path: str, profiling_results: Optional[Dict] = None):
        self.excel_file_path = Path(excel_file_path)
        self.profiling_results = profiling_results
        self.app = None
        self.wb = None
        self.wb_openpyxl = None
        
        # Optimization tracking
        self.issues_found = []
        self.optimizations_applied = []
        self.cleanup_stats = defaultdict(int)
        
        print(f"🔧 Excel Model Optimizer initialized for: {self.excel_file_path.name}")
    
    def __enter__(self):
        """Context manager entry"""
        self._open_workbooks()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit"""
        self._close_workbooks()
    
    def _open_workbooks(self):
        """Open both xlwings and openpyxl workbooks"""
        try:
            # xlwings for operations and VBA
            self.app = xw.App(visible=False)
            self.wb = self.app.books.open(self.excel_file_path)
            
            # openpyxl for detailed formula analysis
            self.wb_openpyxl = openpyxl.load_workbook(self.excel_file_path)
            
            print(f"✅ Workbooks opened for analysis and optimization")
            
        except Exception as e:
            print(f"❌ Failed to open Excel file: {e}")
            raise
    
    def _close_workbooks(self):
        """Close both workbooks"""
        if self.wb_openpyxl:
            self.wb_openpyxl.close()
        if self.wb:
            self.wb.close()
        if self.app:
            self.app.quit()
    
    def audit_macros(self) -> Dict:
        """
        Step 1: Macro audit and cleanup
        """
        print("\n🔍 Step 1: Macro Audit and Cleanup...")
        
        macro_audit = {
            'vba_modules': {},
            'procedures': {},
            'unused_procedures': [],
            'error_handling_coverage': {},
            'optimization_suggestions': []
        }
        
        try:
            # Access VBA project
            vb_project = self.wb.api.VBProject
            
            for component in vb_project.VBComponents:
                module_name = component.Name
                code_module = component.CodeModule
                line_count = code_module.CountOfLines
                
                print(f"   Analyzing VBA module: {module_name}")
                
                # Read all code
                if line_count > 0:
                    code_text = code_module.Lines(1, line_count)
                    
                    # Find procedures
                    procedures = self._extract_vba_procedures(code_text)
                    
                    # Analyze error handling
                    error_handling = self._analyze_error_handling(code_text)
                    
                    macro_audit['vba_modules'][module_name] = {
                        'line_count': line_count,
                        'procedure_count': len(procedures),
                        'procedures': procedures,
                        'has_error_handling': error_handling['has_error_handling'],
                        'error_coverage_percentage': error_handling['coverage_percentage']
                    }
                    
                    # Check for unused procedures
                    for proc_name in procedures:
                        if not self._is_procedure_used(proc_name, code_text):
                            macro_audit['unused_procedures'].append({
                                'module': module_name,
                                'procedure': proc_name,
                                'recommendation': 'Consider removing if truly unused'
                            })
                            
                            self.issues_found.append(OptimizationIssue(
                                category="Macro Cleanup",
                                severity="LOW",
                                location=f"{module_name}.{proc_name}",
                                description="Potentially unused VBA procedure",
                                recommendation="Review and remove if not needed",
                                impact="Small file size reduction"
                            ))
            
            # Generate optimization suggestions
            if len(macro_audit['unused_procedures']) > 0:
                macro_audit['optimization_suggestions'].append(
                    f"Found {len(macro_audit['unused_procedures'])} potentially unused procedures"
                )
            
            print(f"   ✅ Macro audit complete: {len(macro_audit['vba_modules'])} modules analyzed")
            
        except Exception as e:
            print(f"   ⚠️  VBA analysis limited: {e}")
            macro_audit['error'] = str(e)
        
        return macro_audit
    
    def _extract_vba_procedures(self, code_text: str) -> List[str]:
        """Extract procedure names from VBA code"""
        procedures = []
        
        # Regex patterns for Sub and Function declarations
        patterns = [
            r'^\s*(Public|Private)?\s*Sub\s+(\w+)',
            r'^\s*(Public|Private)?\s*Function\s+(\w+)'
        ]
        
        for line in code_text.split('\n'):
            for pattern in patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    procedure_name = match.group(2)
                    if procedure_name not in procedures:
                        procedures.append(procedure_name)
        
        return procedures
    
    def _analyze_error_handling(self, code_text: str) -> Dict:
        """Analyze error handling coverage in VBA code"""
        lines = code_text.split('\n')
        total_procedures = len(re.findall(r'^\s*(Sub|Function)\s+\w+', code_text, re.IGNORECASE | re.MULTILINE))
        
        error_handling_indicators = [
            'On Error Resume Next',
            'On Error GoTo',
            'Err.Number',
            'Error Handling',
            'Try',
            'Catch'
        ]
        
        procedures_with_error_handling = 0
        has_error_handling = any(indicator in code_text for indicator in error_handling_indicators)
        
        if has_error_handling and total_procedures > 0:
            # Rough estimate of coverage
            procedures_with_error_handling = min(total_procedures, 
                                               sum(1 for indicator in error_handling_indicators 
                                                   if indicator in code_text))
        
        coverage_percentage = (procedures_with_error_handling / max(total_procedures, 1)) * 100
        
        return {
            'has_error_handling': has_error_handling,
            'coverage_percentage': round(coverage_percentage, 1),
            'total_procedures': total_procedures
        }
    
    def _is_procedure_used(self, procedure_name: str, code_text: str) -> bool:
        """Check if a procedure is called anywhere in the code"""
        # Simple check - look for procedure name calls
        call_patterns = [
            rf'\b{procedure_name}\s*\(',  # Function call with parentheses
            rf'\bCall\s+{procedure_name}\b',  # Explicit Call statement
            rf'^\s*{procedure_name}\s*$'  # Simple procedure call
        ]
        
        for pattern in call_patterns:
            if re.search(pattern, code_text, re.IGNORECASE | re.MULTILINE):
                return True
        
        return False
    
    def analyze_named_ranges(self) -> Dict:
        """
        Step 2: Named range analysis and cleanup
        """
        print("\n📏 Step 2: Named Range Analysis...")
        
        named_range_analysis = {
            'total_named_ranges': 0,
            'redundant_ranges': [],
            'complex_ranges': [],
            'unused_ranges': [],
            'scope_issues': [],
            'optimization_potential': 0
        }
        
        try:
            # Get all named ranges
            named_ranges = {}
            
            for name in self.wb.names:
                try:
                    range_name = name.name
                    refers_to = name.refers_to
                    scope = "Workbook"  # xlwings doesn't easily distinguish scope
                    
                    named_ranges[range_name] = {
                        'refers_to': refers_to,
                        'scope': scope,
                        'is_complex': self._is_complex_named_range(refers_to)
                    }
                    
                    # Check if range is overly complex
                    if named_ranges[range_name]['is_complex']:
                        named_range_analysis['complex_ranges'].append({
                            'name': range_name,
                            'formula': refers_to,
                            'recommendation': 'Consider simplifying or replacing with direct references'
                        })
                        
                        self.issues_found.append(OptimizationIssue(
                            category="Named Range",
                            severity="MEDIUM",
                            location=range_name,
                            description="Complex named range formula",
                            recommendation="Simplify formula or use direct cell references",
                            impact="Improved calculation speed"
                        ))
                    
                except Exception as e:
                    continue
            
            named_range_analysis['total_named_ranges'] = len(named_ranges)
            
            # Check for unused named ranges
            unused_ranges = self._find_unused_named_ranges(named_ranges)
            named_range_analysis['unused_ranges'] = unused_ranges
            
            # Check for redundant ranges (pointing to same location)
            redundant_groups = self._find_redundant_named_ranges(named_ranges)
            named_range_analysis['redundant_ranges'] = redundant_groups
            
            optimization_potential = len(unused_ranges) + sum(len(group) - 1 for group in redundant_groups)
            named_range_analysis['optimization_potential'] = optimization_potential
            
            print(f"   ✅ Named range analysis: {len(named_ranges)} ranges, {optimization_potential} optimization opportunities")
            
        except Exception as e:
            print(f"   ⚠️  Named range analysis failed: {e}")
            named_range_analysis['error'] = str(e)
        
        return named_range_analysis
    
    def _is_complex_named_range(self, formula: str) -> bool:
        """Determine if a named range formula is overly complex"""
        if not formula:
            return False
        
        complexity_indicators = [
            'INDIRECT(',
            'OFFSET(',
            'INDEX(',
            'MATCH(',
            len(formula) > 100,  # Long formulas
            formula.count(',') > 5,  # Many parameters
            formula.count('(') > 3,  # Nested functions
        ]
        
        return sum(1 for indicator in complexity_indicators if indicator) >= 2
    
    def _find_unused_named_ranges(self, named_ranges: Dict) -> List[Dict]:
        """Find named ranges that aren't used anywhere"""
        unused_ranges = []
        
        # Get all formulas from all sheets
        all_formulas = []
        for sheet in self.wb_openpyxl.worksheets:
            for row in sheet.iter_rows():
                for cell in row:
                    if cell.data_type == 'f' and cell.value:
                        all_formulas.append(str(cell.value))
        
        all_formula_text = ' '.join(all_formulas).upper()
        
        for range_name in named_ranges.keys():
            if range_name.upper() not in all_formula_text:
                unused_ranges.append({
                    'name': range_name,
                    'refers_to': named_ranges[range_name]['refers_to'],
                    'recommendation': 'Consider removing if truly unused'
                })
                
                self.issues_found.append(OptimizationIssue(
                    category="Named Range",
                    severity="LOW",
                    location=range_name,
                    description="Unused named range",
                    recommendation="Remove to reduce file size",
                    impact="Cleaner workbook structure"
                ))
        
        return unused_ranges
    
    def _find_redundant_named_ranges(self, named_ranges: Dict) -> List[List[str]]:
        """Find named ranges that refer to the same location"""
        refers_to_groups = defaultdict(list)
        
        for range_name, info in named_ranges.items():
            refers_to_groups[info['refers_to']].append(range_name)
        
        # Return groups with more than one named range
        redundant_groups = [group for group in refers_to_groups.values() if len(group) > 1]
        
        for group in redundant_groups:
            self.issues_found.append(OptimizationIssue(
                category="Named Range",
                severity="LOW",
                location=', '.join(group),
                description="Multiple named ranges pointing to same location",
                recommendation="Keep one and remove others",
                impact="Reduced confusion and file size"
            ))
        
        return redundant_groups
    
    def analyze_formulas(self) -> Dict:
        """
        Step 3: Formula analysis and optimization
        """
        print("\n📐 Step 3: Formula Analysis and Optimization...")
        
        formula_analysis = {
            'total_formulas': 0,
            'volatile_functions': {},
            'array_formulas': [],
            'circular_references': [],
            'complex_formulas': [],
            'redundant_calculations': [],
            'optimization_opportunities': 0
        }
        
        try:
            total_formulas = 0
            volatile_count = defaultdict(int)
            
            # Volatile functions to look for
            volatile_functions = ['NOW()', 'TODAY()', 'INDIRECT(', 'OFFSET(', 'RAND()', 'RANDBETWEEN(']
            
            for sheet_name in self.wb_openpyxl.sheetnames:
                sheet = self.wb_openpyxl[sheet_name]
                sheet_formulas = {}
                
                print(f"   Analyzing formulas in: {sheet_name}")
                
                for row in sheet.iter_rows():
                    for cell in row:
                        if cell.data_type == 'f' and cell.value:
                            total_formulas += 1
                            formula = str(cell.value).upper()
                            cell_address = f"{sheet_name}!{cell.coordinate}"
                            
                            # Check for volatile functions
                            for volatile_func in volatile_functions:
                                if volatile_func in formula:
                                    volatile_count[volatile_func] += 1
                            
                            # Check for array formulas
                            if cell.data_type == 'f' and hasattr(cell, 'array_formula') and cell.array_formula:
                                formula_analysis['array_formulas'].append({
                                    'location': cell_address,
                                    'formula': cell.value,
                                    'recommendation': 'Consider optimizing if causing performance issues'
                                })
                            
                            # Check for overly complex formulas
                            if self._is_complex_formula(formula):
                                formula_analysis['complex_formulas'].append({
                                    'location': cell_address,
                                    'formula': cell.value,
                                    'length': len(formula),
                                    'nesting_level': formula.count('('),
                                    'recommendation': 'Consider breaking into multiple steps'
                                })
                                
                                self.issues_found.append(OptimizationIssue(
                                    category="Formula Complexity",
                                    severity="MEDIUM",
                                    location=cell_address,
                                    description="Overly complex formula",
                                    recommendation="Break into simpler components",
                                    impact="Improved readability and maintainability"
                                ))
                            
                            # Store for redundancy checking
                            if formula not in sheet_formulas:
                                sheet_formulas[formula] = []
                            sheet_formulas[formula].append(cell_address)
                
                # Find redundant formulas in this sheet
                for formula, locations in sheet_formulas.items():
                    if len(locations) > 1:
                        formula_analysis['redundant_calculations'].append({
                            'formula': formula,
                            'locations': locations,
                            'count': len(locations),
                            'recommendation': 'Consider calculating once and referencing'
                        })
            
            formula_analysis['total_formulas'] = total_formulas
            formula_analysis['volatile_functions'] = dict(volatile_count)
            
            # Calculate optimization opportunities
            optimization_opportunities = (
                sum(volatile_count.values()) +
                len(formula_analysis['complex_formulas']) +
                len(formula_analysis['redundant_calculations'])
            )
            formula_analysis['optimization_opportunities'] = optimization_opportunities
            
            print(f"   ✅ Formula analysis: {total_formulas} formulas, {optimization_opportunities} optimization opportunities")
            
        except Exception as e:
            print(f"   ⚠️  Formula analysis failed: {e}")
            formula_analysis['error'] = str(e)
        
        return formula_analysis
    
    def _is_complex_formula(self, formula: str) -> bool:
        """Determine if a formula is overly complex"""
        complexity_score = 0
        
        # Length-based complexity
        if len(formula) > 200:
            complexity_score += 2
        elif len(formula) > 100:
            complexity_score += 1
        
        # Nesting complexity
        nesting_level = formula.count('(')
        if nesting_level > 5:
            complexity_score += 2
        elif nesting_level > 3:
            complexity_score += 1
        
        # Function count
        function_count = len(re.findall(r'\w+\(', formula))
        if function_count > 5:
            complexity_score += 1
        
        # Reference complexity
        reference_count = len(re.findall(r'[A-Z]+[0-9]+', formula))
        if reference_count > 10:
            complexity_score += 1
        
        return complexity_score >= 3
    
    def analyze_workbook_structure(self) -> Dict:
        """
        Step 4: Comprehensive workbook structure analysis
        """
        print("\n🏗️  Step 4: Workbook Structure Analysis...")
        
        structure_analysis = {
            'file_size_analysis': {},
            'worksheet_analysis': {},
            'conditional_formatting': {},
            'data_types': {},
            'range_references': {},
            'optimization_recommendations': []
        }
        
        # File size analysis
        file_size = self.excel_file_path.stat().st_size
        structure_analysis['file_size_analysis'] = {
            'total_size_bytes': file_size,
            'total_size_mb': round(file_size / (1024 * 1024), 2)
        }
        
        # Analyze each worksheet
        for sheet in self.wb.sheets:
            sheet_name = sheet.name
            print(f"   Analyzing structure of: {sheet_name}")
            
            try:
                used_range = sheet.used_range
                if used_range:
                    # Get dimensions
                    last_row = used_range.last_cell.row
                    last_col = used_range.last_cell.column
                    
                    # Analyze data types and potential issues
                    data_type_analysis = self._analyze_data_types(sheet, used_range)
                    range_ref_analysis = self._analyze_range_references(sheet)
                    
                    structure_analysis['worksheet_analysis'][sheet_name] = {
                        'used_range': f"A1:{used_range.last_cell.address}",
                        'dimensions': f"{last_row} rows x {last_col} columns",
                        'total_cells': last_row * last_col,
                        'data_types': data_type_analysis,
                        'range_references': range_ref_analysis
                    }
                    
                    # Check for excessive worksheet size
                    if last_row > 10000 or last_col > 100:
                        self.issues_found.append(OptimizationIssue(
                            category="Worksheet Structure",
                            severity="MEDIUM",
                            location=sheet_name,
                            description=f"Large worksheet dimensions: {last_row}x{last_col}",
                            recommendation="Consider data optimization or splitting",
                            impact="Improved performance and memory usage"
                        ))
                
            except Exception as e:
                structure_analysis['worksheet_analysis'][sheet_name] = {
                    'error': str(e)
                }
        
        print(f"   ✅ Structure analysis complete")
        
        return structure_analysis
    
    def _analyze_data_types(self, sheet, used_range) -> Dict:
        """Analyze data types in a worksheet"""
        data_types = defaultdict(int)
        potential_issues = []
        
        # Sample cells to avoid performance issues
        sample_size = min(1000, used_range.count)
        
        for i in range(sample_size):
            try:
                row = (i % used_range.last_cell.row) + 1
                col = (i // used_range.last_cell.row) + 1
                
                if row <= used_range.last_cell.row and col <= used_range.last_cell.column:
                    cell = sheet.range(row, col)
                    
                    if cell.value is not None:
                        value_type = type(cell.value).__name__
                        data_types[value_type] += 1
                        
                        # Check for text-as-numbers (common issue)
                        if isinstance(cell.value, str):
                            try:
                                float(cell.value)
                                potential_issues.append(f"Text-as-number at {cell.address}")
                            except ValueError:
                                pass
            except:
                continue
        
        return {
            'type_distribution': dict(data_types),
            'sample_size': sample_size,
            'potential_issues': potential_issues[:10]  # Limit to first 10
        }
    
    def _analyze_range_references(self, sheet) -> Dict:
        """Analyze range references for optimization opportunities"""
        full_column_refs = 0
        full_row_refs = 0
        external_refs = 0
        
        try:
            # Sample formulas to check for inefficient references
            if hasattr(sheet, 'used_range') and sheet.used_range:
                for row in range(1, min(sheet.used_range.last_cell.row + 1, 100)):  # Sample first 100 rows
                    for col in range(1, min(sheet.used_range.last_cell.column + 1, 26)):  # Sample first 26 columns
                        try:
                            cell = sheet.range(row, col)
                            if cell.formula:
                                formula = str(cell.formula)
                                
                                # Check for full column references (A:A, B:B, etc.)
                                if re.search(r'[A-Z]+:[A-Z]+', formula):
                                    full_column_refs += 1
                                
                                # Check for full row references (1:1, 2:2, etc.)
                                if re.search(r'\d+:\d+', formula):
                                    full_row_refs += 1
                                
                                # Check for external references
                                if '[' in formula or '!' in formula and sheet.name not in formula:
                                    external_refs += 1
                        except:
                            continue
        except:
            pass
        
        issues = []
        if full_column_refs > 5:
            issues.append("Many full column references found - consider using specific ranges")
        if external_refs > 10:
            issues.append("Many external references - consider data consolidation")
        
        return {
            'full_column_references': full_column_refs,
            'full_row_references': full_row_refs,
            'external_references': external_refs,
            'optimization_issues': issues
        }
    
    def generate_optimization_report(self) -> Dict:
        """
        Generate comprehensive optimization report
        """
        print("\n📊 Generating Optimization Report...")
        
        # Run all analysis steps
        macro_audit = self.audit_macros()
        named_range_analysis = self.analyze_named_ranges()
        formula_analysis = self.analyze_formulas()
        structure_analysis = self.analyze_workbook_structure()
        
        # Compile comprehensive report
        optimization_report = {
            'report_generated': datetime.now().isoformat(),
            'excel_file': str(self.excel_file_path),
            'summary': {
                'total_issues_found': len(self.issues_found),
                'high_priority_issues': len([i for i in self.issues_found if i.severity == 'HIGH']),
                'medium_priority_issues': len([i for i in self.issues_found if i.severity == 'MEDIUM']),
                'low_priority_issues': len([i for i in self.issues_found if i.severity == 'LOW'])
            },
            'detailed_analysis': {
                'macro_audit': macro_audit,
                'named_range_analysis': named_range_analysis,
                'formula_analysis': formula_analysis,
                'structure_analysis': structure_analysis
            },
            'all_issues': [issue.to_dict() for issue in self.issues_found],
            'priority_recommendations': self._generate_priority_recommendations(),
            'cleanup_stats': dict(self.cleanup_stats)
        }
        
        print(f"   ✅ Optimization report generated: {len(self.issues_found)} issues identified")
        
        return optimization_report
    
    def _generate_priority_recommendations(self) -> List[Dict]:
        """Generate prioritized list of optimization recommendations"""
        priority_recs = []
        
        # Group issues by category and severity
        issue_groups = defaultdict(list)
        for issue in self.issues_found:
            issue_groups[issue.category].append(issue)
        
        # Generate category-based recommendations
        for category, issues in issue_groups.items():
            high_priority = [i for i in issues if i.severity == 'HIGH']
            medium_priority = [i for i in issues if i.severity == 'MEDIUM']
            
            if high_priority:
                priority_recs.append({
                    'priority': 1,
                    'category': category,
                    'issue_count': len(high_priority),
                    'recommendation': f"Address {len(high_priority)} high-priority {category.lower()} issues",
                    'expected_impact': "Significant performance improvement"
                })
            
            if medium_priority:
                priority_recs.append({
                    'priority': 2,
                    'category': category,
                    'issue_count': len(medium_priority),
                    'recommendation': f"Review {len(medium_priority)} medium-priority {category.lower()} issues",
                    'expected_impact': "Moderate performance improvement"
                })
        
        return sorted(priority_recs, key=lambda x: x['priority'])

In [ ]:
def main():
    """
    Main function for Excel Model Optimizer
    """
    excel_file = "project_finance_lease_model.xlsm"
    
    if not Path(excel_file).exists():
        print(f"❌ Excel file not found: {excel_file}")
        return
    
    # Load profiling results if available
    profiling_file = Path(excel_file).stem + "_profiling_results.json"
    profiling_results = None
    
    if Path(profiling_file).exists():
        with open(profiling_file, 'r') as f:
            profiling_results = json.load(f)
        print(f"📂 Loaded profiling results from: {profiling_file}")
    
    # Run optimization analysis
    with ExcelModelOptimizer(excel_file, profiling_results) as optimizer:
        report = optimizer.generate_optimization_report()
        
        # Save optimization report
        output_file = Path(excel_file).stem + "_optimization_report.json"
        with open(output_file, 'w') as f:
            json.dump(report, f, indent=2)
        
        print(f"💾 Optimization report saved to: {output_file}")
        
        # Print summary
        print("\n" + "="*60)
        print("📋 OPTIMIZATION SUMMARY")
        print("="*60)
        print(f"Issues Found: {report['summary']['total_issues_found']}")
        print(f"  - High Priority: {report['summary']['high_priority_issues']}")
        print(f"  - Medium Priority: {report['summary']['medium_priority_issues']}")
        print(f"  - Low Priority: {report['summary']['low_priority_issues']}")
        
        if report['priority_recommendations']:
            print(f"\n🎯 TOP RECOMMENDATIONS:")
            for rec in report['priority_recommendations'][:5]:
                print(f"  {rec['priority']}. {rec['recommendation']}")

In [ ]:
if __name__ == "__main__":
    main()